In [9]:
from typing import List, Dict, Tuple
from sklearn import linear_model, neighbors
import pandas as pd
import numpy as np
import random

U = [
    'cloud_commercial',
    'cloud_own',
    'pc',
    'mobile'
]

C = [
    'wifi_public_no_passwd',
    'wifi_public_passwd',
    'cellular',
    'wifi_priv_known',
]

C_to_lvl = {v:i for i,v in enumerate(C)}

where_combined = [
    'mobile',
    'wifi_pc',
    'wifi_cloud',
    'cellular_cloud'
]

MAX_SEC_LEVEL = 4

def security_filter(dx: int, is_vpn: bool) -> Tuple[List[str], List[str]]:
    Ua: List[str] = []
    Ca: List[str] = []

    if dx == 0:
        Ua = U[:]
        Ca = C[:]

    else:
        if dx == MAX_SEC_LEVEL:
            Ua = U[MAX_SEC_LEVEL-1]
        else:
            Ua = U[dx - 1:]
            Ca = C[dx - int(is_vpn):]

    return Ca, Ua


def linear_regression(x: List[float], y: List[float]) -> linear_model.LinearRegression:
    regression = linear_model.LinearRegression()
    regression.fit(np.array(x).reshape(-1,1), np.array(y).reshape(-1,1))

    return regression


def knn(x: List[float], y: List[float]) -> neighbors.KNeighborsRegressor:
    neigh = neighbors.KNeighborsRegressor(n_neighbors=3, weights='distance')
    neigh.fit(np.array(x).reshape(-1,1), np.array(y).reshape(-1,1))

    return neigh

def nn(x: List[float], y: List[float]) -> linear_model.LinearRegression:
    regression = linear_model.LinearRegression()
    regression.fit(np.array(x).reshape(-1,1), np.array(y).reshape(-1,1))

    return regression

def svm(x: List[float], y: List[float]) -> linear_model.LinearRegression:
    regression = linear_model.LinearRegression()
    regression.fit(np.array(x).reshape(-1,1), np.array(y).reshape(-1,1))

    return regression

def naive_bayes(x: List[float], y: List[float]) -> linear_model.LinearRegression:
    regression = linear_model.LinearRegression()
    regression.fit(np.array(x).reshape(-1,1), np.array(y).reshape(-1,1))

    return regression

def random_forest(x: List[float], y: List[float]) -> linear_model.LinearRegression:
    regression = linear_model.LinearRegression()
    regression.fit(np.array(x).reshape(-1,1), np.array(y).reshape(-1,1))

    return regression

def logistic_regression(x: List[float], y: List[float]) -> linear_model.LinearRegression:
    regression = linear_model.LinearRegression()
    regression.fit(np.array(x).reshape(-1,1), np.array(y).reshape(-1,1))

    return regression

def decision_tree(x: List[float], y: List[float]) -> linear_model.LinearRegression:
    regression = linear_model.LinearRegression()
    regression.fit(np.array(x).reshape(-1,1), np.array(y).reshape(-1,1))

    return regression

def gradient_boosting(x: List[float], y: List[float]) -> linear_model.LinearRegression:
    regression = linear_model.LinearRegression()
    regression.fit(np.array(x).reshape(-1,1), np.array(y).reshape(-1,1))

    return regression

def context_model_predict(jx: float, models: Dict):
    # rx = {
    #     'mobile': {
    #         'battery': models['mobile']['battery'].predict(jx),
    #         'time': models['mobile']['time'].predict(jx)
    #     },
    #     'wifi_pc': {
    #         'battery': models['wifi_pc']['battery'].predict(jx),
    #         'time': models['wifi_pc']['time'].predict(jx)
    #     },
    #     'wifi_cloud': {
    #         'battery': models['wifi_cloud']['battery'].predict(jx),
    #         'time': models['wifi_cloud']['time'].predict(jx)
    #     },
    #     'cellular_cloud': {
    #         'battery': models['cellular_cloud']['battery'].predict(jx),
    #         'time': models['cellular_cloud']['time'].predict(jx)
    #     }
    # }
    rx = {
        how: {
            'battery': models[how]['battery'].predict([[jx]])[0][0],
            'time': models[how]['time'].predict([[jx]])[0][0]
        } for how in where_combined
    }

    return rx

def where_name(c: str, u: str) -> str:

    return c.split('_')[0] + '_' + u.split('_')[0]

def choose_optimal_service(rx: Dict, s: Tuple[List[str], List[str]], Cr: List[str], priority: str) -> Tuple[str, str, float, float]:
    possible_combinations = []

    for c in s[0]:
        if c in Cr:
            for u in s[1]:
                if u == 'mobile':
                    continue
                if u == 'pc' and c.startswith('wifi'):
                    where = where_name(c, u)
                    possible_combinations.append((c, u, rx[where]['battery'], rx[where]['time']))
                    continue # let's check next possible connection
                if u.startswith('cloud') and (c == 'cellular' or c.startswith('wifi')):
                    where = where_name(c, u)
                    possible_combinations.append((c, u, rx[where]['battery'], rx[where]['time']))
                    continue

    possible_combinations.append(('none', 'mobile', rx['mobile']['battery'], rx['mobile']['time']))

    if priority == 'security':
        pass # possible_combinations already ordered by security

    elif priority == 'battery':
        # since sort is stable than the choice will contain the best security for lowest battery drain
        possible_combinations.sort(key=lambda x: x[2], reverse=True)

    elif priority == 'speed':
        # since sort is stable than the choice will contain the best security for fastest computation
        possible_combinations.sort(key=lambda x: x[3], reverse=True)

    elif priority == 'general':
        # best battery and time in regard to security restrictions
        possible_combinations.sort(key=lambda x: x[2], reverse=True)
        possible_combinations.sort(key=lambda x: x[3], reverse=True)

    elif priority == 'general-not-secure':
        possible_combinations.sort(key=lambda x: x[2])
        possible_combinations.sort(key=lambda x: x[3])
        return possible_combinations[0]

    elif priority == 'speed-not-secure':
        random.shuffle(possible_combinations)
        possible_combinations.sort(key=lambda x: x[3], reverse=True)

    elif priority == 'true-random':
        random.shuffle(possible_combinations)

    return possible_combinations[-1]

In [10]:
data = pd.read_csv('./learning_data.csv')

In [11]:
models = {
    where: {
        'battery': linear_regression(list(data[data['where'] == where]['our_metric']), list(data[data['where'] == where]['battery_used'])),
        'time': linear_regression(list(data[data['where'] == where]['our_metric']), list(data[data['where'] == where]['time_needed']))
    } for where in where_combined
}

In [12]:
def execute_task(jx: int, dx: int, Cr: List[str], priority: str, isVPN: bool):
    rx = context_model_predict(jx, models)
    Ca, Ua = security_filter(dx, isVPN)
    return choose_optimal_service(rx, (Ca, Ua), Cr, priority)

In [13]:
from itertools import combinations
from functools import reduce
Cr = [
    'wifi_public_no_passwd',
    'wifi_public_passwd',
    'cellular',
    'wifi_priv_known',
]

tasks = list(range(5000, 100000 +1, 5000))
# tasks_with_file = [
#     ('540x40',3183),
#     ('580x680',8463),
#     ('600x120',8493),
#     ('720x960',83883),
#     ('960x480',33574),
#     ('960x1280',98774),
#     ('1200x1200',17473),
#     ('1600x1200',22769),
# ]

possible_Cr = reduce(lambda a, b: a + b, [list(combinations(Cr, i)) for i in range(4+1)])
sec_range = list(range(MAX_SEC_LEVEL+1))
possible_priority = ['general', 'general-not-secure', 'battery', 'speed', 'speed-not-secure', 'true-random']
# vpn

In [14]:
def achieved_security_lvl(connection: str):
    if connection == 'none':
        return 4
    else:
        return C_to_lvl[connection]

In [17]:
with open('tests.csv', 'w') as fp:
    fp.write('task_metric;data_security;with_vpn;available_connections;achieved_security;priority;chosen_connection;chosen_service;predicted_battery;predicted_time\n')

In [18]:
# task_metric = 5000
# data_security = 0
# available_connections = ('wifi_public_no_passwd', 'wifi_public_passwd', 'cellular')
# security_lvl = 1
# priority = 'general'

with open('tests_files.csv', 'a') as fp:
    for with_vpn in [False, True]:
        print('With VPN:', with_vpn)
        for index, data_security in enumerate(sec_range):
            print('%.2f' % (index/(len(sec_range)-1)))
            for available_connections in possible_Cr:
                for priority in possible_priority:
                    for task_metric in tasks:
                        chosen_connection, chosen_service, predicted_battery, predicted_time = execute_task(task_metric, data_security, available_connections, priority, with_vpn)
                        achieved_security = achieved_security_lvl(chosen_connection)
                        row = f'{task_metric};{data_security};{with_vpn};{available_connections};{achieved_security};{priority};{chosen_connection};{chosen_service};{predicted_battery};{predicted_time}\n'
                        fp.write(row)

With VPN: False
0.00
0.25
0.50
0.75
1.00
With VPN: True
0.00
0.25
0.50
0.75
1.00


In [230]:
asdasd = pd.read_csv('tests.csv', sep=';')

In [231]:
import ast

ast.literal_eval(asdasd['available_connections'][0])[0]

'wifi_public_no_passwd'

In [274]:
execute_task(55000, 0, Cr, 'general-not-secure', False)

('wifi_public_no_passwd', 'pc', 0.00626796074171361, 2.1362177397235502)